# Algoritmo KNN paso a paso
Este cuaderno guía el desarrollo de un ejemplo completo del algoritmo *k*-Nearest Neighbors (KNN) en clasificación supervisada. Seguiremos un flujo que incluye la creación del script `ejemplo_knn.py`, la generación y exploración de datos sintéticos, la configuración de hiperparámetros, el entrenamiento documentado línea a línea y el análisis de resultados.

## 1. Configurar dependencias y preparar el script
A continuación importamos las bibliotecas que usaremos en el resto del flujo y creamos el archivo `ejemplo_knn.py`. La celda siguiente usa la *magic* `%%writefile` para dejar una estructura mínima sobre la cual iremos añadiendo el código documentado.

In [1]:
%%writefile ejemplo_knn.py
"""Ejemplo completo del algoritmo KNN con documentación paso a paso."""

import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt


def main() -> None:
    """La lógica principal se completará en secciones posteriores."""
    raise NotImplementedError("El contenido se escribirá en los siguientes pasos del cuaderno.")


if __name__ == "__main__":
    main()

Overwriting ejemplo_knn.py


**Explicación línea a línea del script generado:**
- `"""Ejemplo completo..."""` deja un docstring que describe el propósito del programa.
- `import numpy as np` importa NumPy para manipular arrays numéricos de forma eficiente.
- `import pandas as pd` trae pandas para construir estructuras tabulares y resúmenes.
- `from sklearn.datasets import make_classification` habilita la función que crea un dataset sintético para clasificación.
- `from sklearn.model_selection import train_test_split` permitirá separar los datos en entrenamiento y prueba.
- `from sklearn.preprocessing import StandardScaler` aporta el escalado estándar indispensable en KNN.
- `from sklearn.neighbors import KNeighborsClassifier` importa el estimador principal que entrenaremos.
- `from sklearn.metrics import accuracy_score, classification_report, confusion_matrix` reúne métricas para evaluar el rendimiento del modelo.
- `import matplotlib.pyplot as plt` posibilita visualizaciones y gráficos.
- `def main() -> None:` define el punto de entrada; se actualizará con la lógica completa.
- `raise NotImplementedError(...)` recuerda que el contenido se completará más adelante en el cuaderno.
- `if __name__ == "__main__":` asegura la ejecución de `main()` solo cuando el archivo se ejecuta como script.

## 2. Generar datos sintéticos y explorarlos
Usaremos `make_classification` para sintetizar ejemplos bidimensionales que permitan visualizar fácilmente las fronteras de decisión de KNN. Las celdas de esta sección muestran cómo se construyen las variables, se resumen las estadísticas y se representa la nube de puntos inicial.

In [ ]:
SEED = 42
feature_names = ["feature_1", "feature_2"]
features, labels = make_classification(
    n_samples=400,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    class_sep=1.2,
    random_state=SEED,
)
df = pd.DataFrame(features, columns=feature_names)
df["target"] = labels
df.head()

**Detalles de la generación del dataset:**
- `SEED = 42` fija una semilla para reproducir los resultados.
- `feature_names = [...]` define etiquetas para las columnas numéricas.
- `features, labels = make_classification(...` crea 400 observaciones bidimensionales; destaca que `n_redundant=0` evita variables redundantes y `class_sep=1.2` separa clases para un reto moderado.
- `random_state=SEED` garantiza la reproducibilidad del muestreo sintético.
- `df = pd.DataFrame(...)` transforma los arrays en una tabla pandas con nombres legibles.
- `df["target"] = labels` anexa la columna objetivo categórica.
- `df.head()` muestra las primeras filas para inspeccionar la estructura resultante.

In [ ]:
df.describe(include="all").transpose()

**Resumen estadístico:**
- `df.describe(include="all")` calcula estadísticas básicas para cada columna.
- `.transpose()` organiza los resultados en filas para facilitar la lectura.
- El conteo (`count`) confirma las 400 observaciones; la media (`mean`) y desviación estándar (`std`) muestran la escala de cada rasgo, útil para anticipar la necesidad de escalado.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
scatter = ax.scatter(
    df["feature_1"],
    df["feature_2"],
    c=df["target"],
    cmap="coolwarm",
    edgecolor="black",
    alpha=0.75,
)
ax.set_title("Distribución inicial de las clases sintetizadas")
ax.set_xlabel("feature_1")
ax.set_ylabel("feature_2")
legend = ax.legend(*scatter.legend_elements(), title="Clase")
ax.add_artist(legend)
plt.show()

**Lectura del gráfico de dispersión:**
- `fig, ax = plt.subplots(...)` crea la figura y el eje donde graficaremos.
- `ax.scatter(...)` pinta cada observación; `c=df["target"]` colorea según la clase y `edgecolor="black"` mejora el contraste.
- `ax.set_title`, `set_xlabel`, `set_ylabel` nombran el gráfico y los ejes.
- `ax.legend(*scatter.legend_elements(), ...)` genera una leyenda automáticamente a partir de los valores de la columna objetivo.
- `plt.show()` asegura que la figura se renderice en el cuaderno.

## 3. Dividir datos en entrenamiento y prueba
Dividimos el dataset en subconjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo sobre ejemplos no vistos. Escogemos una proporción 80/20 con la misma semilla aleatoria establecida antes.

In [ ]:
X = df[feature_names].values
y = df["target"].values
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=SEED,
)
X_train.shape, X_test.shape

**Desglose de la partición:**
- `X = df[feature_names].values` extrae una matriz NumPy con las dos variables predictoras.
- `y = df["target"].values` obtiene el vector de etiquetas para entrenamiento supervisado.
- `train_test_split(...)` divide los datos: `test_size=0.2` deja 20% para prueba, `stratify=y` mantiene la proporción de clases en cada conjunto y `random_state=SEED` hace reproducible la partición.
- `X_train.shape, X_test.shape` confirma las dimensiones de los subconjuntos generados.

## 4. Configurar hiperparámetros del KNN
Antes de entrenar debemos normalizar los datos y decidir los hiperparámetros clave del modelo: número de vecinos (`n_neighbors`), esquema de ponderación (`weights`) y métrica de distancia (`metric`). Se incluye una celda para el escalado y otra que fija los valores elegidos justificándolos.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Por qué escalamos:**
- `scaler = StandardScaler()` instancia el estandarizador que centra la media en 0 y ajusta la desviación estándar a 1.
- `scaler.fit_transform(X_train)` aprende la media y desviación a partir del conjunto de entrenamiento y transforma los datos.
- `scaler.transform(X_test)` aplica la misma transformación al conjunto de prueba sin recalcular parámetros para evitar fuga de información.

In [ ]:
knn_params = {
    "n_neighbors": 5,
    "weights": "distance",
    "metric": "minkowski",
    "p": 2,
    "algorithm": "auto",
    "leaf_size": 30,
}
knn_params

**Justificación de los hiperparámetros:**
- `n_neighbors=5` promedia la información de cinco vecinos, suavizando el contorno de decisión y equilibrando sesgo-varianza.
- `weights="distance"` da más peso a los vecinos más cercanos para reducir decisiones ruidosas en zonas mixtas.
- `metric="minkowski"` junto con `p=2` equivale a la distancia euclidiana clásica; permite cambiar a Manhattan (`p=1`) si fuera necesario.
- `algorithm="auto"` deja que scikit-learn elija la estrategia de búsqueda más eficiente según el tamaño del dataset.
- `leaf_size=30` controla el tamaño de los nodos en estructuras KDTree/BallTree; su valor por defecto funciona bien para datasets medianos.

## 5. Implementar y documentar el entrenamiento KNN línea a línea
Reescribimos `ejemplo_knn.py` incorporando toda la lógica del pipeline con comentarios que detallan cada instrucción. La estructura reproduce las etapas vistas en las secciones anteriores para que el script pueda ejecutarse de forma autónoma.

In [ ]:
%%writefile ejemplo_knn.py
"""Ejemplo completo del algoritmo KNN con documentación paso a paso."""

from typing import Dict, Tuple

import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

SEED = 42
FEATURE_NAMES = ["feature_1", "feature_2"]


def build_dataset(seed: int) -> pd.DataFrame:
    """Genera un dataset sintético reproducible con dos características informativas."""
    # Generamos la matriz de características y el vector objetivo usando make_classification.
    features, labels = make_classification(
        n_samples=400,
        n_features=2,
        n_redundant=0,
        n_informative=2,
        n_clusters_per_class=1,
        class_sep=1.2,
        random_state=seed,
    )
    # Convertimos las características en un DataFrame con nombres amigables.
    df = pd.DataFrame(features, columns=FEATURE_NAMES)
    # Añadimos la columna objetivo con las etiquetas de clase.
    df["target"] = labels
    return df


def split_dataset(
    df: pd.DataFrame, seed: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Divide el dataset en entrenamiento y prueba conservando la proporción de clases."""
    # Separamos las columnas predictoras de la columna objetivo.
    X = df[FEATURE_NAMES].values
    y = df["target"].values
    # Realizamos la partición estratificada para mantener el balance de clases.
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        stratify=y,
        random_state=seed,
    )
    return X_train, X_test, y_train, y_test


def scale_features(
    X_train: np.ndarray, X_test: np.ndarray
) -> Tuple[np.ndarray, np.ndarray, StandardScaler]:
    """Escala los subconjuntos con StandardScaler y devuelve el objeto entrenado."""
    # Instanciamos el estandarizador para centrar y escalar las variables.
    scaler = StandardScaler()
    # Ajustamos el escalador usando solo el conjunto de entrenamiento y transformamos los valores.
    X_train_scaled = scaler.fit_transform(X_train)
    # Aplicamos la misma transformación a los datos de prueba.
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler


def instantiate_model(params: Dict[str, object]) -> KNeighborsClassifier:
    """Crea el clasificador KNN con los hiperparámetros indicados."""
    # Construimos el estimador pasando el diccionario de hiperparámetros con desempaquetado.
    model = KNeighborsClassifier(**params)
    return model


def train_model(
    model: KNeighborsClassifier, X_train: np.ndarray, y_train: np.ndarray
) -> KNeighborsClassifier:
    """Ajusta el modelo KNN y devuelve la instancia entrenada."""
    # Llamamos a fit para almacenar los ejemplos de entrenamiento en la estructura interna del modelo.
    model.fit(X_train, y_train)
    return model


def evaluate_model(
    model: KNeighborsClassifier,
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> Tuple[float, float, str, np.ndarray]:
    """Calcula métricas de rendimiento sobre entrenamiento y prueba."""
    # Generamos predicciones sobre el conjunto de entrenamiento para medir sobreajuste potencial.
    train_predictions = model.predict(X_train)
    # Generamos predicciones sobre el conjunto de prueba para estimar generalización.
    test_predictions = model.predict(X_test)
    # Calculamos la exactitud en entrenamiento como referencia.
    train_accuracy = accuracy_score(y_train, train_predictions)
    # Calculamos la exactitud en prueba para medir el desempeño real.
    test_accuracy = accuracy_score(y_test, test_predictions)
    # Producimos un informe detallado con precisión, recall y f1-score por clase.
    detailed_report = classification_report(y_test, test_predictions)
    # Construimos la matriz de confusión para analizar aciertos y errores por clase.
    confusion = confusion_matrix(y_test, test_predictions)
    return train_accuracy, test_accuracy, detailed_report, confusion


def plot_decision_boundary(
    model: KNeighborsClassifier,
    scaler: StandardScaler,
    df: pd.DataFrame,
) -> None:
    """Grafica la frontera de decisión aprendida."""
    # Determinamos los límites del plano añadiendo un margen alrededor de los datos.
    x_min, x_max = df["feature_1"].min() - 0.5, df["feature_1"].max() + 0.5
    y_min, y_max = df["feature_2"].min() - 0.5, df["feature_2"].max() + 0.5
    # Creamos una malla regular con paso fino para evaluar las predicciones del modelo.
    grid_x, grid_y = np.meshgrid(
        np.linspace(x_min, x_max, 300),
        np.linspace(y_min, y_max, 300),
    )
    # Reorganizamos la malla en una matriz de coordenadas y la escalamos con los parámetros aprendidos.
    grid_points = np.c_[grid_x.ravel(), grid_y.ravel()]
    grid_points_scaled = scaler.transform(grid_points)
    # Calculamos las predicciones del modelo sobre cada punto de la malla.
    grid_predictions = model.predict(grid_points_scaled).reshape(grid_x.shape)
    # Definimos un mapa de colores suave para representar las clases.
    cmap = ListedColormap(["#ff9999", "#99ccff"])
    # Construimos la figura y el eje para el gráfico de frontera.
    fig, ax = plt.subplots(figsize=(7, 5))
    # Pintamos la superficie con las predicciones usando contourf.
    ax.contourf(grid_x, grid_y, grid_predictions, alpha=0.4, cmap=cmap)
    # Dibujamos los puntos reales por encima para comparar con la frontera.
    scatter = ax.scatter(
        df["feature_1"],
        df["feature_2"],
        c=df["target"],
        cmap="coolwarm",
        edgecolor="black",
        alpha=0.75,
    )
    # Añadimos título y etiquetas descriptivas a los ejes.
    ax.set_title("Frontera de decisión estimada por KNN")
    ax.set_xlabel("feature_1")
    ax.set_ylabel("feature_2")
    # Creamos la leyenda derivada de los elementos del diagrama de dispersión.
    legend = ax.legend(*scatter.legend_elements(), title="Clase")
    ax.add_artist(legend)
    # Mostramos el gráfico en pantalla.
    plt.show()


def main() -> None:
    """Ejecuta el flujo completo de entrenamiento, evaluación y visualización."""
    # Fijamos la semilla de NumPy para reproducir cualquier operación aleatoria adicional.
    np.random.seed(SEED)
    # Construimos el dataset sintético que se usará durante todo el flujo.
    dataset = build_dataset(seed=SEED)
    # Dividimos el dataset en subconjuntos de entrenamiento y prueba.
    X_train, X_test, y_train, y_test = split_dataset(dataset, seed=SEED)
    # Escalamos los subconjuntos obteniendo también el escalador entrenado.
    X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)
    # Definimos los hiperparámetros que regirán las predicciones del modelo KNN.
    params = {
        "n_neighbors": 5,
        "weights": "distance",
        "metric": "minkowski",
        "p": 2,
        "algorithm": "auto",
        "leaf_size": 30,
    }
    # Creamos el clasificador KNN con los hiperparámetros seleccionados.
    knn = instantiate_model(params)
    # Ajustamos el clasificador usando los datos escalados de entrenamiento.
    knn = train_model(knn, X_train_scaled, y_train)
    # Evaluamos el rendimiento tanto en entrenamiento como en prueba.
    train_acc, test_acc, report, confusion = evaluate_model(
        knn,
        X_train_scaled,
        y_train,
        X_test_scaled,
        y_test,
    )
    # Mostramos la métrica de exactitud en consola para comparar ambos conjuntos.
    print(f"Exactitud entrenamiento: {train_acc:.3f}")
    print(f"Exactitud prueba: {test_acc:.3f}")
    # Presentamos el informe detallado con precisión, recall y f1-score.
    print("\nInforme de clasificación:\n" + report)
    # Imprimimos la matriz de confusión para analizar errores específicos.
    print("Matriz de confusión:\n", confusion)
    # Visualizamos la frontera de decisión obtenida por el modelo entrenado.
    plot_decision_boundary(knn, scaler, dataset)


if __name__ == "__main__":
    main()

**Puntos clave del script actualizado:**
- `build_dataset` encapsula la creación del dataset sintético con comentarios que justifican cada parámetro de `make_classification`.
- `split_dataset` y `scale_features` separan y estandarizan los datos asegurando reproducibilidad (`random_state=SEED`).
- `instantiate_model` y `train_model` muestran explícitamente cómo se construye y ajusta el clasificador usando el diccionario `params`.
- `evaluate_model` calcula exactitud, reporte y matriz de confusión explicando el motivo de cada línea.
- `plot_decision_boundary` crea la malla, transforma los puntos con el `StandardScaler` y visualiza la frontera aprendida.
- `main` orquesta el flujo completo imprimiendo métricas y llamando a la función de visualización.

## 6. Evaluar el modelo con métricas detalladas
Ejecutamos las funciones creadas para entrenar el modelo en el entorno interactivo y analizamos métricas como exactitud, matriz de confusión e informe de clasificación.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

knn_model = KNeighborsClassifier(**knn_params)
knn_model.fit(X_train_scaled, y_train)
train_predictions = knn_model.predict(X_train_scaled)
test_predictions = knn_model.predict(X_test_scaled)
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Exactitud entrenamiento: {train_accuracy:.3f}")
print(f"Exactitud prueba: {test_accuracy:.3f}")
print("\nInforme de clasificación:\n", classification_report(y_test, test_predictions))

**Interpretación de la celda anterior:**
- `from sklearn.metrics import ConfusionMatrixDisplay` habilita la visualización matricial que emplearemos luego.
- `knn_model = KNeighborsClassifier(**knn_params)` recrea el clasificador con los hiperparámetros discutidos.
- `fit`, `predict` sobre entrenamiento y prueba calculan las etiquetas estimadas para evaluar rendimiento y sobreajuste.
- `accuracy_score` ofrece la exactitud en ambos subconjuntos; comparar ambas cifras ayuda a detectar overfitting.
- `classification_report(...)` genera precisión, recall y f1 por clase, útil para diagnosticar desbalanceos.

In [ ]:
cm = confusion_matrix(y_test, test_predictions)
ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.show()

**Lectura de la matriz de confusión:**
- `confusion_matrix(y_test, test_predictions)` cruza las clases reales y predichas.
- `ConfusionMatrixDisplay(cm).plot(...)` dibuja la matriz en escala de azules para resaltar aciertos y errores.
- `plt.show()` asegura que la figura se renderice al ejecutar el cuaderno.

## 7. Visualizar fronteras de decisión y análisis final
Para comprender la influencia de los hiperparámetros graficamos la frontera de decisión del modelo y debatimos cómo afecta cada elección sobre el comportamiento observado.

In [ ]:
import importlib
import ejemplo_knn

importlib.reload(ejemplo_knn)
ejemplo_knn.plot_decision_boundary(knn_model, scaler, df)

**Análisis de la frontera de decisión:**
- `importlib.reload(ejemplo_knn)` garantiza que usemos la versión del script generada en esta sesión.
- `plot_decision_boundary(knn_model, scaler, df)` aplica el escalador aprendido a la malla de puntos y dibuja la clasificación que produce el modelo entrenado.
- El contorno resultante muestra regiones suaves gracias a `n_neighbors=5`; un valor menor generaría fronteras más irregulares, mientras que uno mayor las haría demasiado lisas.
- El esquema `weights="distance"` provoca transiciones más graduales cerca de los límites porque otorga más influencia a los vecinos inmediatos.
- Cambiar `metric` a Manhattan (`p=1`) inclinaría las fronteras hacia líneas con ángulos rectos, evidenciando la importancia de este hiperparámetro.

### Conclusiones
El pipeline demuestra cómo cada hiperparámetro de KNN influye en la frontera de decisión y en las métricas finales. El script `ejemplo_knn.py` queda listo para ejecutarse de forma independiente, reproduciendo los pasos explicados en el cuaderno y facilitando la experimentación con distintos valores de `n_neighbors`, `weights` y `metric`.